# Text Mining Analyse op Incidentbeschrijvingen

In dit notebook voeren we een eenvoudige textmining-analyse uit op incidentbeschrijvingen.
Doel:
- Per incidentcategorie inzicht krijgen in de meest voorkomende woorden (woordfrequentie).
- Per incidentcategorie de meest **onderscheidende** woorden bepalen met behulp van TF-IDF.

Bestand: `textmining_beschrijving-Incident.csv`
Belangrijke kolommen:
- `incident nummer`
- `Incident categorie`
- `Beschrijving incident`

In [42]:
!pip install pandas numpy scikit-learn
!pip install nltk


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


**Imports & instellingen**

In [43]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

**Data inladen & eerste verkenning**

In [44]:
# CSV inladen
bestand = "textmining_beschrijving-Incident.csv"

df = pd.read_csv(bestand, sep=",")
df.head()

,incident nummer,Incident categorie,Beschrijving incident
0,1065298,2e lijns (Doorgezette incidenten),Nieuwe KA: ctxh21-02 geen beschikbare servers
1,1124826,Hardware / Werkplekbeheer,TC loopt met enige regelmaat vast of hapert
2,1156332,2e lijns (Doorgezette incidenten),Kaartlezer wordt niet offline ondersteund.
3,1183663,Functioneel beheer,"iTask, rechten herstellen"
4,1183923,Functioneel beheer,"iPortal, alarmnummers opnemen in kwaliteitspor..."


**Basis schoonmaak en controle**

In [45]:
# Kolomnamen checken
df.columns

# Verwijder lege beschrijvingen
df = df.dropna(subset=["Beschrijving incident"])

# Aantal rijen na opschoning
df.shape

(25803, 3)

**Tekst per categorie samenvoegen**

In [46]:
# Alle teksten per incidentcategorie samenvoegen tot één grote string
cat_text = (
    df.groupby("Incident categorie")["Beschrijving incident"]
      .apply(lambda s: " ".join(s.astype(str)))
)

cat_text

Incident categorie
1e lijns beheer (Helpdesk)           Excel openen door direct een bestand te openen...
2e lijns (Doorgezette incidenten)    Nieuwe KA: ctxh21-02 geen beschikbare servers ...
3e lijns (Technische incidenten)     Comez Aanvraag voor Mobile Inlog en WS Private...
Functioneel beheer                   iTask, rechten herstellen iPortal, alarmnummer...
Hardware / Werkplekbeheer            TC loopt met enige regelmaat vast of hapert wi...
Telefonie / Communicatie             ophalen gsm noodprocedure telefonie telefoon w...
Name: Beschrijving incident, dtype: object

**Eigen stopwoordenlijst omdat scikit-learn versie géén stop_words="dutch" ondersteund volgens de foutmelding**

In [47]:
# Nederlandse stopwoordenlijst
dutch_stopwords = [
    "de","het","een","en","van","op","aan","in","met","voor","naar","dat","die","dit",
    "is","als","bij","door","kan","kunnen","zal","zullen","te","tot","ook","maar","om",
    "niet","wel","dan","er","me","mijn","je","jouw","zijn","haar","hun","of","u","we",
    "wij","jullie","ze","zij","wordt","worden","heb","hebt","heeft","hebben"
]

**Stopwoordenlijst automatisch slimmer maken voor betere inzichten**

In [48]:
from collections import Counter
import re

# 1. Tekst opschonen
def tokenize(text):
    return re.findall(r"\b[a-zA-Z]{2,}\b", text.lower())

# Alle woorden uit hele dataset
all_words = []
for t in df["Beschrijving incident"]:
    all_words.extend(tokenize(str(t)))

word_freq = Counter(all_words)

# 2. Detecteer woorden die veel te vaag zijn (top 50 woorden)
most_common_words = [w for w, c in word_freq.most_common(80)]

# 3. Filter automatisch ruis via regels
auto_stopwords = [
    w for w in most_common_words
    if len(w) <= 3                        # te korte woorden
    or w in ["graag", "svp"]              # servicedesk-vookeurruis
    or w.isdigit()                        # nummers
]

# 4. Voeg jouw specifieke ruiswoorden toe
custom_noise_words = [
    "dv", "cs", "ka", "ww",
    "wire", "solutions",          # organisatie-naam
    "amsterdam", "connect",       # vaak oninformatief in context
    "crm", "tokencode",           # afhankelijk van wat je wil verwijderen
]

# 5. Combineer met je basis-stopwoorden
final_stopwords = list(set(dutch_stopwords + auto_stopwords + custom_noise_words))

print("Aantal stopwoorden:", len(final_stopwords))
print("Voorbeeld:", list(final_stopwords)[:40])


Aantal stopwoorden: 74
Voorbeeld: ['door', 'erp', 'te', 'tc', 'graag', 'dan', 'tokencode', 'maar', 'mijn', 'heb', 'solutions', 'en', 'die', 'jouw', 'dit', 'zullen', 'in', 'ook', 'wel', 'zijn', 'zij', 'na', 'er', 'haar', 'ww', 'kan', 'je', 'svp', 'me', 'met', 'pas', 'wire', 'of', 'jullie', 'de', 'kunnen', 'hebt', 'op', 'amsterdam', 'heeft']


**Woordfrequentie per categorie**

In [49]:
# CountVectorizer voor woordfrequenties
vectorizer = CountVectorizer(
    stop_words=final_stopwords,
    max_df=0.85,
    min_df=2
)
X_counts = vectorizer.fit_transform(cat_text.values)
woorden = vectorizer.get_feature_names_out()

X_counts_array = X_counts.toarray()

N = 20  # aantal topwoorden per categorie

for cat, row in zip(cat_text.index, X_counts_array):
    idx = np.argsort(row)[::-1][:N]

    print("\n==============================")
    print(f"📌 {cat} – Top {N} meest voorkomende woorden")
    print("==============================")

    for i in idx:
        print(f"{woorden[i]:20s} {row[i]}")


📌 1e lijns beheer (Helpdesk) – Top 20 meest voorkomende woorden
printer              94
code                 89
safeword             87
outlook              50
reset                46
wachtwoord           44
activeren            41
handtekening         27
foutmelding          22
inlog                19
printen              15
zetten               14
inlogproblemen       14
niks                 14
chipsoft             14
word                 12
mfc                  12
mailbox              12
citrix               12
zwart                11

📌 2e lijns (Doorgezette incidenten) – Top 20 meest voorkomende woorden
outlook              898
doorverbonden        273
safewordcode         248
printers             247
printer              239
account              236
handtekening         225
wachtwoord           218
rechten              199
token                199
vergeten             194
activeren            192
openen               191
gevraagd             186
printen              181
starten 

**Resultaten naar een DataFrame**

_Als je de topwoorden later in Excel of Word wilt plakken:_

In [50]:
def top_words_per_category_count(X, feature_names, categories, topn=20):
    records = []
    X_arr = X.toarray()

    for cat, row in zip(categories, X_arr):
        idx = np.argsort(row)[::-1][:topn]
        for rank, i in enumerate(idx, start=1):
            records.append({
                "Categorie": cat,
                "Rang": rank,
                "Woord": feature_names[i],
                "Frequentie": int(row[i])
            })
    return pd.DataFrame(records)

df_top_counts = top_words_per_category_count(X_counts, woorden, cat_text.index, topn=20)
df_top_counts.head()

,Categorie,Rang,Woord,Frequentie
0,1e lijns beheer (Helpdesk),1,printer,94
1,1e lijns beheer (Helpdesk),2,code,89
2,1e lijns beheer (Helpdesk),3,safeword,87
3,1e lijns beheer (Helpdesk),4,outlook,50
4,1e lijns beheer (Helpdesk),5,reset,46


**TF-IDF hottopics per categorie**

In [51]:
# TF-IDF vectorizer
tfidf = TfidfVectorizer(
    stop_words=final_stopwords,
    max_df=0.85,
    min_df=2
)

X_tfidf = tfidf.fit_transform(cat_text.values)
woorden_tfidf = tfidf.get_feature_names_out()

X_tfidf_array = X_tfidf.toarray()

N = 15  # aantal TF-IDF hottopics per categorie

for cat, row in zip(cat_text.index, X_tfidf_array):
    idx = np.argsort(row)[::-1][:N]

    print("\n==============================")
    print(f"🔥 {cat} – TF-IDF hottopics (Top {N})")
    print("==============================")

    for i in idx:
        print(f"{woorden_tfidf[i]:30s} {row[i]:.4f}")



🔥 1e lijns beheer (Helpdesk) – TF-IDF hottopics (Top 15)
safeword                       0.6124
printer                        0.4134
code                           0.3914
outlook                        0.2199
reset                          0.2023
wachtwoord                     0.1935
activeren                      0.1803
handtekening                   0.1375
foutmelding                    0.0968
inlog                          0.0836
mfc                            0.0713
mailbox                        0.0713
zetten                         0.0713
printen                        0.0660
chipsoft                       0.0616

🔥 2e lijns (Doorgezette incidenten) – TF-IDF hottopics (Top 15)
outlook                        0.4775
safewordcode                   0.2111
doorverbonden                  0.1681
gevraagd                       0.1583
handtekening                   0.1385
printers                       0.1313
printer                        0.1271
account                        0.1255
tok

**TF-IDF resultaten ook naar DataFrame**

In [52]:
def top_words_per_category_tfidf(X, feature_names, categories, topn=15):
    records = []
    X_arr = X.toarray()

    for cat, row in zip(categories, X_arr):
        idx = np.argsort(row)[::-1][:topn]
        for rank, i in enumerate(idx, start=1):
            records.append({
                "Categorie": cat,
                "Rang": rank,
                "Woord": feature_names[i],
                "TFIDF_score": float(row[i])
            })
    return pd.DataFrame(records)

df_top_tfidf = top_words_per_category_tfidf(X_tfidf, woorden_tfidf, cat_text.index, topn=15)
df_top_tfidf.head()

,Categorie,Rang,Woord,TFIDF_score
0,1e lijns beheer (Helpdesk),1,safeword,0.612432
1,1e lijns beheer (Helpdesk),2,printer,0.413420
2,1e lijns beheer (Helpdesk),3,code,0.391430
3,1e lijns beheer (Helpdesk),4,outlook,0.219904
4,1e lijns beheer (Helpdesk),5,reset,0.202312


## Interpretatie van de resultaten

Per incidentcategorie zijn de belangrijkste woorden (frequentie) en hottopics (TF-IDF) bepaald.

- **Eenvoudige woordfrequentie** laat zien welke termen het vaakst voorkomen binnen een categorie.
- **TF-IDF** benadrukt juist woorden die typisch zijn voor een categorie vergeleken met andere categorieën.

Voorbeeld van mogelijke interpretatie per categorie:
- 1e lijns beheer (Helpdesk): veel voorkomende woorden rond wachtwoorden, inloggen, mail, telefoons.
- 2e lijns (Doorgezette incidenten): meer technisch inhoudelijke termen zoals ERP, database, timeouts, autorisatie.
- 3e lijns (Technische incidenten): netwerk, servers, VPN, storage, etc.
- Functioneel beheer: processen, rechten, formulieren, applicatiespecificieke termen.
- Hardware / Werkplekbeheer: laptop, scherm, toetsenbord, docking, etc.
- Telefonie / Communicatie: VoIP, vaste lijn, bereikbaarheid, headset, softphone.

Deze inzichten kun je koppelen aan:
- Welke problemen horen bij welke lijn?
- Waar overlappen onderwerpen tussen 1e en 2e lijn?
- Welke terugkerende thema's vragen structurele maatregelen (training, documentatie, systeemwijzigingen)?

**Hoe interpreteer je TF-IDF in het rapport?**

TF-IDF laat zien welke woorden uniek of typisch zijn voor een categorie.

Bijvoorbeeld:
- Een woord met TF-IDF 0.8+ → bijna uitsluitend in die categorie
- Een woord met TF-IDF 0.5–0.7 → sterk onderscheidend
- Een woord met TF-IDF 0.2–0.4 → relevant, maar gedeeltelijk gedeeld met andere categorieën
- Een woord met TF-IDF < 0.1 → weinig onderscheidend